## 1° PROYECTO INDIVIDUAL MLOps -1er parte

### ETL

### 1) Importación de librerías

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import skew
from math import sqrt
from numpy import mean, var
import copy 
from sklearn import preprocessing
import json

### 2) Lectura de archivo csv como dataframe

In [204]:
df=pd.read_csv('movies_dataset.csv') 
df.head(2) 

C:\Users\Martina_2\AppData\Local\Temp\ipykernel_7460\1469590950.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('movies_dataset.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [205]:
# Analizamos estructura del dataset
df.shape

(45466, 24)

### 3) Transformaciones

#### a) Expansión de columnas anidadas

##### Colecciones

In [206]:
import ast
import re

# Definimos una expresión regular para extraer el nombre de la colección
pattern = r"'name': '([^']*)'"

# Extraemos el nombre de la colección utilizando la expresión regular
df['collection_name'] = df['belongs_to_collection'].str.extract(pattern, flags=re.IGNORECASE)

# Imprimimos el DataFrame con la columna normalizada
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection


In [207]:
df.collection_name.unique()

array(['Toy Story Collection', nan, 'Grumpy Old Men Collection', ...,
       'Ducobu Collection', 'Mister Blot Collection',
       'Red Lotus Collection'], dtype=object)

In [208]:
df.collection_name.nunique()

1644

In [209]:
# Crear una nueva columna para los registros duplicados
df['collection_individual'] = df['collection_name'].str.split(', ')

# Duplicar los registros
df = df.explode('collection_individual').reset_index(drop=True)

# Eliminar la columna original "collection_name"
df.drop('collection_name', axis=1, inplace=True)

In [210]:
df.collection_individual.unique()

array(['Toy Story Collection', nan, 'Grumpy Old Men Collection', ...,
       'Ducobu Collection', 'Mister Blot Collection',
       'Red Lotus Collection'], dtype=object)

In [211]:
df.collection_individual.nunique()

1667

##### Géneros

In [212]:
# Convertimos la cadena JSON en una lista de diccionarios
df['genres'] = df['genres'].apply(ast.literal_eval)

# Función personalizada para extraer los nombres de los géneros
def extract_genre_names(genres_list):
    genre_names = [genre['name'] for genre in genres_list]
    return ', '.join(genre_names)

# Aplicamos la función personalizada a cada elemento de la columna "genres"
df['genre'] = df['genres'].apply(extract_genre_names)

# Imprimimos el DataFrame con la columna de géneros normalizados
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_individual,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,"Animation, Comedy, Family"
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,"Adventure, Fantasy, Family"
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection,"Romance, Comedy"
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN,"Comedy, Drama, Romance"
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection,Comedy


In [213]:
df.genre.unique()

array(['Animation, Comedy, Family', 'Adventure, Fantasy, Family',
       'Romance, Comedy', ..., 'Crime, Comedy, Action, Family',
       'Action, Mystery, Thriller, Horror',
       'Family, Animation, Romance, Comedy'], dtype=object)

In [214]:
df.genre.nunique()

4069

In [215]:
# Crear una nueva columna para los registros duplicados
df['genre_individual'] = df['genre'].str.split(', ')

# Duplicar los registros
df = df.explode('genre_individual').reset_index(drop=True)

# Eliminar la columna original "genre"
df.drop('genre', axis=1, inplace=True)

In [216]:
df.genre_individual.unique()

array(['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance',
       'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History',
       'Science Fiction', 'Mystery', 'War', 'Foreign', '', 'Music',
       'Documentary', 'Western', 'TV Movie', 'Carousel Productions',
       'Vision View Entertainment', 'Telescene Film Group Productions',
       'Aniplex', 'GoHands', 'BROSTA TV',
       'Mardock Scramble Production Committee', 'Sentai Filmworks',
       'Odyssey Media', 'Pulser Productions', 'Rogue State', 'The Cartel'],
      dtype=object)

In [217]:
df.genre_individual.nunique()

33

##### Compañías productoras

In [218]:
# Función personalizada para normalizar la columna "production_companies"
def normalize_production_companies(value):
    try:
        pattern = r"'name':\s+'(.*?)'"
        matches = re.findall(pattern, value)
        return ', '.join(matches)
    except (TypeError, AttributeError):
        return ''

# Aplicamos la función personalizada a cada elemento de la columna "production_companies"
df['company'] = df['production_companies'].apply(normalize_production_companies)

# Imprimimos el DataFrame con la columna de compañías de producción normalizada
df.head()


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_individual,genre_individual,company
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios
2,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Family,Pixar Animation Studios
3,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,"TriStar Pictures, Teitler Film, Interscope Com..."
4,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Fantasy,"TriStar Pictures, Teitler Film, Interscope Com..."


In [219]:
df.company.unique()

array(['Pixar Animation Studios',
       'TriStar Pictures, Teitler Film, Interscope Communications',
       'Warner Bros., Lancaster Gate', ...,
       'Westdeutscher Rundfunk (WDR), Working Title Films, 20th Century Fox Television, CanWest Global Communications',
       'Sine Olivia', 'Yermoliev'], dtype=object)

In [220]:
df.company.nunique()

22580

In [221]:
# Crear una nueva columna para los registros duplicados
df['company_individual'] = df['company'].str.split(', ')

# Duplicar los registros
df = df.explode('company_individual').reset_index(drop=True)

# Eliminar la columna original "company"
df.drop('company', axis=1, inplace=True)

In [222]:
df.company_individual.unique()

array(['Pixar Animation Studios', 'TriStar Pictures', 'Teitler Film', ...,
       'Pirie Productions', 'Sine Olivia', 'Yermoliev'], dtype=object)

In [223]:
df.company_individual.nunique()

23350

##### Países productores

In [224]:
# Función personalizada para normalizar la columna "production_countries"
def normalize_production_countries(value):
    try:
        pattern = r"'name':\s+'(.*?)'"
        matches = re.findall(pattern, value)
        return ', '.join(matches)
    except (TypeError, AttributeError):
        return ''

# Aplicamos la función personalizada a cada elemento de la columna "production_countries"
df['country'] = df['production_countries'].apply(normalize_production_countries)

# Imprimimos el DataFrame con la columna de países de producción normalizada
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,collection_individual,genre_individual,company_individual,country
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America
2,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Family,Pixar Animation Studios,United States of America
3,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,TriStar Pictures,United States of America
4,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,Teitler Film,United States of America


In [225]:
df.country.unique()

array(['United States of America', 'Germany, United States of America',
       'United Kingdom, United States of America', ...,
       'Poland, Czech Republic, Slovakia', 'Cuba, Germany, Spain',
       'Egypt, Italy, United States of America'], dtype=object)

In [226]:
df.country.nunique()

2387

In [227]:
# Crear una nueva columna para los registros duplicados
df['country_individual'] = df['country'].str.split(', ')

# Duplicar los registros
df = df.explode('country_individual').reset_index(drop=True)

# Eliminar la columna original "country"
df.drop('country', axis=1, inplace=True)

In [228]:
df.country_individual.unique()

array(['United States of America', 'Germany', 'United Kingdom', 'France',
       'Italy', 'Spain', 'China', 'Australia', 'South Africa', 'Canada',
       '', 'Switzerland', 'Belgium', 'Japan', 'Iran', 'Netherlands',
       'Hong Kong', 'Tunisia', 'Ireland', 'Dominican Republic', 'Croatia',
       'Russia', 'Macedonia', 'Austria', 'Taiwan', 'New Zealand',
       'Mexico', 'Poland', 'Peru', 'Cuba', 'Liechtenstein', 'Denmark',
       'Portugal', 'Finland', 'Sweden', 'Argentina', 'Iceland',
       'South Korea', 'Serbia', 'Hungary', 'Czech Republic', 'India',
       'Brazil', 'Greece', 'Congo', 'Senegal', 'Burkina Faso', 'Romania',
       'Philippines', 'Vietnam', 'Trinidad and Tobago', 'Bulgaria',
       'Chile', 'Norway', 'Kazakhstan', 'Algeria', 'Luxembourg',
       'Georgia', 'Ukraine', 'Botswana', 'Aruba', 'Israel', 'Turkey',
       'Ecuador', 'Lebanon', 'Morocco', 'Bosnia and Herzegovina',
       'Bahamas', 'Malaysia', 'Bhutan', 'Jamaica', 'Pakistan', 'Nepal',
       'Thailand', 'Nam

In [229]:
df.country_individual.nunique()

159

##### Lenguajes

In [230]:
# Función personalizada para normalizar la columna "spoken_languages"
def normalize_spoken_languages(value):
    try:
        pattern = r"'name':\s+'(.*?)'"
        matches = re.findall(pattern, value)
        return ', '.join(matches)
    except (TypeError, AttributeError):
        return ''

# Aplicamos la función personalizada a cada elemento de la columna "spoken_languages"
df['spoken_language'] = df['spoken_languages'].apply(normalize_spoken_languages)

# Imprimimos el DataFrame con la columna de lenguajes normalizada
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,collection_individual,genre_individual,company_individual,country_individual,spoken_language
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English
2,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Family,Pixar Animation Studios,United States of America,English
3,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,TriStar Pictures,United States of America,"English, Français"
4,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,Teitler Film,United States of America,"English, Français"


In [231]:
df.spoken_language.unique()

array(['English', 'English, Français', 'English, Español', ...,
       'svenska, Deutsch', 'العربية, Polski', 'Fulfulde, English'],
      dtype=object)

In [232]:
df.spoken_language.nunique()

1842

In [233]:
# Crear una nueva columna para los registros duplicados
df['language_individual'] = df['spoken_language'].str.split(', ')

# Duplicar los registros
df = df.explode('language_individual').reset_index(drop=True)

# Eliminar la columna original "spoken_language"
df.drop('spoken_language', axis=1, inplace=True)

In [234]:
df.language_individual.unique()

array(['English', 'Français', 'Español', 'Deutsch', 'Pусский', 'Latin',
       'Nederlands', '广州话 / 廣州話', '普通话', 'Magyar', '', 'shqip',
       'Italiano', '한국어/조선말', 'فارسی', 'Dansk', '日本語', 'العربية',
       'Hrvatski', 'Bosanski', 'Română', 'Bahasa indonesia',
       'Bahasa melayu', 'svenska', 'עִבְרִית', 'Český', 'Polski',
       'Gaeilge', 'Norsk', 'Slovenčina', 'Tiếng Việt', 'Português',
       'हिन्दी', 'Català', 'Íslenska', 'Afrikaans', 'Srpski', 'বাংলা',
       'Wolof', 'Cymraeg', 'ภาษาไทย', 'Latviešu', 'Kiswahili',
       'български език', 'ελληνικά', 'Türkçe', 'suomi', 'Esperanto',
       'Український', 'ქართული', 'Bokmål', 'No Language', 'euskera',
       'Azərbaycan', 'Malti', 'اردو', 'isiZulu', 'Bamanankan', 'پښتو',
       'Somali', 'ਪੰਜਾਬੀ', 'беларуская мова', 'தமிழ்', 'Galego',
       'Kinyarwanda', 'қазақ', 'Eesti', 'Lietuvi\\x9akai', 'Slovenščina',
       'తెలుగు', 'Fulfulde', '??????', '?????', 'ozbek', 'Hausa'],
      dtype=object)

In [235]:
df.language_individual.nunique()

75

In [236]:
df.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'], inplace = True)
df.head()

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,tagline,title,video,vote_average,vote_count,collection_individual,genre_individual,company_individual,country_individual,language_individual
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English
1,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English
2,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Family,Pixar Animation Studios,United States of America,English
3,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,TriStar Pictures,United States of America,English
4,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,TriStar Pictures,United States of America,Français


In [237]:
df.shape

(419666, 24)

In [238]:
df.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'collection_individual',
       'genre_individual', 'company_individual', 'country_individual',
       'language_individual'],
      dtype='object')

In [239]:
df.rename(columns={ 'collection_individual': 'collection', 
                    'genre_individual': 'genre',
                    'company_individual': 'company',
                    'country_individual': 'country',
                    'language_individual': 'language'
}, inplace=True)
df.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'collection', 'genre', 'company',
       'country', 'language'],
      dtype='object')

#### b) Tratamiento de valores nulos

In [240]:
# Los valores nulos de los campos revenue, budget son rellenados por el número 0

df['revenue'].fillna(0, inplace=True)
df['budget'].fillna(0, inplace=True)
print('Nulos en revenue',df.revenue.isna().sum())
print('Nulos en budget',df.revenue.isna().sum())

Nulos en revenue 0
Nulos en budget 0


#### c) Modificaciones en datos de tipo fecha

In [251]:
# Borramos valores nulos
df = df.dropna(subset = ['release_date'])
df.release_date.isna().sum()

0

In [252]:
# Transformamos el formato de fecha de 'release_date' a 'aaaa-mm-dd' si el formato es incorrecto
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Creamos la columna 'release_year' extrayendo el año de 'release_date'
df['release_year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year
df['release_year'] = df['release_year'].astype('Int64')


In [253]:
df[['release_date','release_year']]

,release_date,release_year
0,1995-10-30,1995
1,1995-10-30,1995
2,1995-10-30,1995
3,1995-12-15,1995
4,1995-12-15,1995
...,...,...
419661,2003-08-01,2003
419662,2003-08-01,2003
419663,2003-08-01,2003
419664,1917-10-21,1917


In [254]:
df.release_year.unique()

<IntegerArray>
[1995, 1996, 1994, 1997, 1976, 1992, 1967, 1993, 1964, 1977,
 ...
 1908, 1897, 1887, 1888, 1890, 1878, 1874, 1906, 1883, 1907]
Length: 135, dtype: Int64

In [ ]:
# Borramos valores nulos
df = df.dropna(subset = ['release_date'])
df.release_date.isna().sum()

In [ ]:
# Borramos valores nulos
df = df.dropna(subset = ['release_year'])
df.release_date.isna().sum()

In [255]:
df.isna().sum()

adult                     0
budget                    0
homepage             291079
id                        0
imdb_id                  61
original_language        96
original_title            0
overview               2488
popularity                0
poster_path             519
release_date              0
revenue                   0
runtime                 544
status                  143
tagline              191768
title                     0
video                     0
vote_average              0
vote_count                0
collection           365817
genre                     0
company                   0
country                   0
language                  0
release_year              0
dtype: int64

In [256]:
df.shape

(419475, 25)

#### d) Nuevas columnas

In [257]:
# Creamos la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, 
# cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0

# Calculamos el retorno de inversión

# Convertimos 'revenue' y 'budget' al tipo numérico (float)
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

# Calculamos el retorno de inversión
df['return'] = df['revenue'] / df['budget']
df['return'].fillna(0, inplace=True)
df['return']

0         12.451801
1         12.451801
2         12.451801
3          4.043035
4          4.043035
            ...    
419661     0.000000
419662     0.000000
419663     0.000000
419664     0.000000
419665     0.000000
Name: return, Length: 419475, dtype: float64

In [258]:
df['return'].isna().sum()

0

In [259]:
df['return'].unique()

array([12.4518011,  4.0430346,  0.       , ...,  1.660765 ,  0.6343965,
        0.2826   ])

#### e) Eliminación de columnas innecesarias

In [260]:
# Eliminamos las columnas que no serán utilizadas, video,imdb_id,adult,original_title,vote_count,poster_path y homepage
df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'vote_count', 'poster_path', 'homepage'], inplace = True)
df.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'collection', 'genre', 'company', 'country', 'language',
       'release_year', 'return'],
      dtype='object')

In [261]:
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,collection,genre,company,country,language,release_year,return
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English,1995,12.451801
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English,1995,12.451801
2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,Toy Story Collection,Family,Pixar Animation Studios,United States of America,English,1995,12.451801
3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,NaN,Adventure,TriStar Pictures,United States of America,English,1995,4.043035
4,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,NaN,Adventure,TriStar Pictures,United States of America,Français,1995,4.043035


In [305]:
df.shape

(419475, 19)

In [263]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 419475 entries, 0 to 419665
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   budget             419475 non-null  int64  
 1   id                 419475 non-null  object 
 2   original_language  419379 non-null  object 
 3   overview           416987 non-null  object 
 4   popularity         419475 non-null  object 
 5   release_date       419475 non-null  object 
 6   revenue            419475 non-null  float64
 7   runtime            418931 non-null  float64
 8   status             419332 non-null  object 
 9   tagline            227707 non-null  object 
 10  title              419475 non-null  object 
 11  vote_average       419475 non-null  float64
 12  collection         53658 non-null   object 
 13  genre              419475 non-null  object 
 14  company            419475 non-null  object 
 15  country            419475 non-null  object 
 16  la

In [264]:
df.tail()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,collection,genre,company,country,language,release_year,return
419661,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,Betrayal,3.8,NaN,Action,American World Pictures,United States of America,English,2003,0.0
419662,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,Betrayal,3.8,NaN,Drama,American World Pictures,United States of America,English,2003,0.0
419663,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,Betrayal,3.8,NaN,Thriller,American World Pictures,United States of America,English,2003,0.0
419664,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,NaN,Satan Triumphant,0.0,NaN,,Yermoliev,Russia,,1917,0.0
419665,0,461257,en,50 years after decriminalisation of homosexual...,0.163015,2017-06-09,0.0,75.0,Released,NaN,Queerama,0.0,NaN,,,United Kingdom,English,2017,0.0


### 4) Exportación a csv

In [265]:
df.to_csv('movies_final.csv', index=False)

### 5) Endpoints

#### a) Películas por mes

In [266]:
def peliculas_mes(mes):
    '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes 
    (nombre del mes, en str, ejemplo 'enero') historicamente
    ''' 
    mes = mes.lower()
    meses = {
    'enero': 1,
    'febrero': 2,
    'marzo': 3,
    'abril': 4,
    'mayo': 5,
    'junio': 6,
    'julio': 7,
    'agosto': 8,
    'septiembre': 9,
    'octubre': 10,
    'noviembre': 11,
    'diciembre': 12}

    mes_numero = meses[mes]

    # Convertir la columna "fecha" a un objeto de tipo fecha
    df['release_date'] = pd.to_datetime(df['release_date'])

    # Tratamos la excepciòn
    try:
        month_filtered = df[df['release_date'].dt.month == mes_numero]
    except (ValueError, KeyError, TypeError):
        return None

    # Filtramos valores duplicados del dataframe y calculamos
    month_unique = month_filtered.drop_duplicates(subset='id')
    respuesta = month_unique.shape[0]

    return {'mes':mes, 'cantidad':respuesta}

In [349]:
peliculas_mes('abril')


{'mes': 'abril', 'cantidad': 3452}

#### b) Películas por día

In [268]:
def peliculas_dia(dia):
    '''Se ingresa el dia y la funcion retorna la cantidad de peliculas que se 
    estrenaron ese dia (de la semana, en str, ejemplo 'lunes') historicamente
    '''
    # Creamos diccionario para normalizar
    days = {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miercoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sabado': 'Saturday',
    'domingo': 'Sunday'}

    day = days[dia.lower()]

    # Filtramos los duplicados del dataframe y calculamos
    lista_peliculas_day = df[df['release_date'].dt.day_name() == day].drop_duplicates(subset='id')
    respuesta = lista_peliculas_day.shape[0]

    return {'dia': dia, 'cantidad': respuesta}

In [350]:
peliculas_dia('sabado')


{'dia': 'sabado', 'cantidad': 5149}

#### c) Métricas por franquicia

In [270]:
def franquicia(franquicia):
    '''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio
    ''' 
    # Filtramos el dataframe
    lista_peliculas_franquicia = df[(df['collection'] == franquicia)].drop_duplicates(subset='id')

    # Calculamos
    cantidad_peliculas_franq = (lista_peliculas_franquicia).shape[0]
    revenue_franq = lista_peliculas_franquicia['revenue'].sum()
    promedio_franq = revenue_franq/cantidad_peliculas_franq

    return {'franquicia':franquicia, 'cantidad':cantidad_peliculas_franq, 'ganancia_total':revenue_franq, 'ganancia_promedio':promedio_franq}

In [351]:
franquicia('Back to the Future Collection') 

{'franquicia': 'Back to the Future Collection',
 'cantidad': 3,
 'ganancia_total': 957637345.0,
 'ganancia_promedio': 319212448.3333333}

#### d) Películas por país

In [272]:
def peliculas_pais(pais):
    '''
    Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo
    '''
    # Filtramos el dataframe y contamos filas
    movies_filtered = df[(df['country'] == pais)]
    movies_unique = movies_filtered.drop_duplicates(subset='id')    
    respuesta = movies_unique.shape[0]
    
    return {'pais':pais, 'cantidad':respuesta}


In [354]:
peliculas_pais('Russia')

{'pais': 'Russia', 'cantidad': 911}

#### e) Métricas por productora

In [274]:
def productoras(productora): 
    '''Ingresas la productora, retornando la ganancia total y la cantidad de peliculas que produjeron
    ''' 
    # Filtramos el dataframe
    lista_peliculas_productoras = df[(df['company'] == productora)].drop_duplicates(subset='id')

    # Calculamos
    cantidad_peliculas_prod = (lista_peliculas_productoras).shape[0]
    revenue_prod = lista_peliculas_productoras['revenue'].sum()

    return {'productora':productora, 'ganancia_total':revenue_prod, 'cantidad':cantidad_peliculas_prod}

In [288]:
productoras('Universal Studios')

{'productora': 'Universal Studios',
 'ganancia_total': 3265387180.0,
 'cantidad': 34}

#### f) Métricas por película

In [347]:
def retorno(pelicula): 
    '''Ingresas la pelicula, retornando la inversion, la ganancia, el retorno y el año en el que se lanzó
    '''   
    # Filtramos el dataframe y Calculamos
    info_pelicula = df[(df['title'] == pelicula)].drop_duplicates(subset='title')
    pelicula_nombre = info_pelicula['title'].iloc[0]
    inversion_pelicula = str(info_pelicula['budget'].iloc[0])
    ganancia_pelicula = str(info_pelicula['revenue'].iloc[0])
    retorno_pelicula = str(info_pelicula['return'].iloc[0])
    year_pelicula = str(info_pelicula['release_year'].iloc[0])

    return {'pelicula':pelicula_nombre, 'inversion':inversion_pelicula, 'ganacia':ganancia_pelicula,'retorno':retorno_pelicula, 'anio':year_pelicula}

In [355]:
retorno('Godzilla')

{'pelicula': 'Godzilla',
 'inversion': '130000000',
 'ganacia': '379014294.0',
 'retorno': '2.9154945692307694',
 'anio': '1998'}